# Manifold GP Semi-Supervised Learning via Precision Matrix on 2D Manifold

## Preamble

In [1]:
import numpy as np
import torch
import gpytorch

%matplotlib widget
import matplotlib.pyplot as plt

from mayavi import mlab
from importlib.resources import files

from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.models.vanilla_gp import VanillaGP
from manifold_gp.utils.generate_truth import groundtruth_from_mesh

## Dataset Preprocessing

### Load

In [2]:
data_path = files('manifold_gp.data').joinpath('dragon.stl')
nodes, faces, truth = groundtruth_from_mesh(data_path)

sampled_x = torch.from_numpy(nodes).float()
sampled_y = torch.from_numpy(truth).float()
(m, n) = sampled_x.shape

num_train = 25
num_test = 1000
normalize_features = False
normalize_labels = True

### Noise Features

In [3]:
noise_sampled_x = 0.01
noisy_x = sampled_x + noise_sampled_x * torch.randn(m, n)

### Trainset & Testset

In [4]:
torch.manual_seed(1337)
rand_idx = torch.randperm(m)
train_idx = rand_idx[:num_train]
train_x, train_y = noisy_x[train_idx, :], sampled_y[train_idx]

noise_train_y = 0.01
train_y += noise_train_y * torch.randn(num_train)

test_idx = rand_idx[num_train:num_train+num_test]
test_x, test_y = sampled_x[test_idx, :], sampled_y[test_idx]

noise_test_y = 0.0
test_y += noise_test_y * torch.randn(num_test)

if normalize_features:
    mu_x, std_x = noisy_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    noisy_x.sub_(mu_x).div_(std_x)
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)
    sampled_y.sub_(mu_y).div_(std_y)

### Move Data to Device

In [5]:
noisy_x, sampled_y = noisy_x.contiguous(), sampled_y.contiguous()
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
noisy_x = noisy_x.to(device)
train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

if normalize_features:
    mu_x, std_x = mu_x.to(device), std_x.to(device)

## Vanilla Pre-Trained

In [6]:
%%capture
model_vanilla = VanillaGP(
    train_x, 
    train_y, 
    gpytorch.likelihoods.GaussianLikelihood(), 
    gpytorch.kernels.ScaleKernel(
        # gpytorch.kernels.RBFKernel()
        gpytorch.kernels.MaternKernel(nu=2.5)
    ) # gpytorch.kernels.RBFKernel(), gpytorch.kernels.RFFKernel(100)
).to(device)
model_vanilla.vanilla_train(lr=1e-1, iter=200, verbose=False)
model_vanilla.likelihood.eval()
model_vanilla.eval()

## Model

In [7]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=2,
        nodes=noisy_x,
        neighbors=10,
        operator="randomwalk",
        method="exact",
        modes=2000,
        ball_scale=5.0,
        ball_decay=0.01,
        prior_bandwidth=True,
    ),
    outputscale_prior=None  # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx, model_vanilla).to(device)

## Train

### Set Model Initial Parameters

In [8]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.epsilon': kernel.base_kernel.epsilon_prior.sample(), # kernel.base_kernel.epsilon_prior.sample()
    'covar_module.base_kernel.lengthscale': 1.0,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

### Manifold Informed Training

In [9]:
model.manifold_informed_train(lr=1e-1, iter=100, norm_step_size=100, verbose=True)
print("NoiseVar: ", likelihood.noise.item(), "SignalVar: ", kernel.outputscale.item(), 
      "Lengthscale: ", kernel.base_kernel.lengthscale.item(), "Epsilon", kernel.base_kernel.epsilon.item())

Iter: 0, Loss: 28.698, NoiseVar: 0.010, SignalVar: 9195467.00000, Lengthscale: 1.000, Epsilon: 0.007
Iter: 1, Loss: 29.378, NoiseVar: 0.009, SignalVar: 9195467.00000, Lengthscale: 0.938, Epsilon: 0.006
Iter: 2, Loss: 28.710, NoiseVar: 0.009, SignalVar: 9195467.00000, Lengthscale: 0.958, Epsilon: 0.007
Iter: 3, Loss: 28.740, NoiseVar: 0.010, SignalVar: 9195467.00000, Lengthscale: 0.979, Epsilon: 0.007
Iter: 4, Loss: 28.946, NoiseVar: 0.010, SignalVar: 9195467.00000, Lengthscale: 0.974, Epsilon: 0.007
Iter: 5, Loss: 28.860, NoiseVar: 0.010, SignalVar: 9195467.00000, Lengthscale: 0.951, Epsilon: 0.007
Iter: 6, Loss: 28.679, NoiseVar: 0.009, SignalVar: 9195467.00000, Lengthscale: 0.918, Epsilon: 0.007
Iter: 7, Loss: 28.658, NoiseVar: 0.009, SignalVar: 9195467.00000, Lengthscale: 0.882, Epsilon: 0.007
Iter: 8, Loss: 28.796, NoiseVar: 0.008, SignalVar: 9195467.00000, Lengthscale: 0.850, Epsilon: 0.007
Iter: 9, Loss: 28.857, NoiseVar: 0.008, SignalVar: 9195467.00000, Lengthscale: 0.828, Epsil

### Extract EigenPairs and Train with Fixed number of Eigenfunctions

In [10]:
kernel.base_kernel.generate_eigenpairs()

exact


In [11]:
model.vanilla_train(lr=1e-1, iter=200, verbose=True)
print("NoiseVar: ", likelihood.noise.item(), "SignalVar: ", kernel.outputscale.item(), 
      "Lengthscale: ", kernel.base_kernel.lengthscale.item(), "Epsilon", kernel.base_kernel.epsilon.item())

Iter: 0, Loss: 0.753, NoiseVar: 0.001, SignalVar: 0.88800, Lengthscale: 0.112, Epsilon: 0.007
Iter: 1, Loss: 0.640, NoiseVar: 0.001, SignalVar: 0.94805, Lengthscale: 0.123, Epsilon: 0.007
Iter: 2, Loss: 0.560, NoiseVar: 0.001, SignalVar: 1.00274, Lengthscale: 0.135, Epsilon: 0.007
Iter: 3, Loss: 0.501, NoiseVar: 0.001, SignalVar: 1.04211, Lengthscale: 0.148, Epsilon: 0.007
Iter: 4, Loss: 0.453, NoiseVar: 0.001, SignalVar: 1.05953, Lengthscale: 0.161, Epsilon: 0.007
Iter: 5, Loss: 0.409, NoiseVar: 0.001, SignalVar: 1.05628, Lengthscale: 0.176, Epsilon: 0.007
Iter: 6, Loss: 0.364, NoiseVar: 0.000, SignalVar: 1.03746, Lengthscale: 0.190, Epsilon: 0.007
Iter: 7, Loss: 0.318, NoiseVar: 0.000, SignalVar: 1.00792, Lengthscale: 0.206, Epsilon: 0.007
Iter: 8, Loss: 0.272, NoiseVar: 0.000, SignalVar: 0.97128, Lengthscale: 0.223, Epsilon: 0.007
Iter: 9, Loss: 0.225, NoiseVar: 0.000, SignalVar: 0.93017, Lengthscale: 0.241, Epsilon: 0.007
Iter: 10, Loss: 0.178, NoiseVar: 0.000, SignalVar: 0.88651, 

### Save/Load Model

In [12]:
torch.save(model.state_dict(), '../outputs/models/2d_manifold.pth')
# state_dict = torch.load('../outputs/models/2d_manifold.pth')
# model.load_state_dict(state_dict)

## Evaluation

In [13]:
%%capture
likelihood.eval()
model.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    model.posterior(test_x, noise=False)
    bump_scale = 1-kernel.base_kernel.bump_function(test_x)

### Vanilla

In [14]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    error = test_y - model.mean("vanilla")
    covar = model.posterior_vanilla.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    rmse = (error.square().mean()).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/error.size(-1)
    model._clear_cache()
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.2733, device='cuda:0')
NLL:  tensor(-0.7858, device='cuda:0')


### Manifold

In [15]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    error = test_y - model.mean("manifold")
    covar = model.posterior_manifold.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    rmse = (error.square().mean()).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/error.size(-1)
    model._clear_cache()
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.2472, device='cuda:0')
NLL:  tensor(-1.0195, device='cuda:0')


### Hybrid

In [16]:
with gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    error = test_y - model.mean("hybrid")
    covar = model.posterior_manifold.lazy_covariance_matrix.evaluate_kernel() 
    + torch.outer(bump_scale,bump_scale) * model.posterior_vanilla.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    rmse = (error.square().mean()).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/error.size(-1)
    model._clear_cache()
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.2435, device='cuda:0')
NLL:  tensor(-1.3117, device='cuda:0')


## Plot

In [17]:
%%capture
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    model.posterior(noisy_x, noise=True)
    mean_manifold = model.mean("hybrid").cpu().numpy()
    mean_vanilla = model.mean("vanilla").cpu().numpy()
    stddev_manifold = model.stddev("hybrid").cpu().numpy()
    stddev_vanilla = model.stddev("vanilla").cpu().numpy()

In [18]:
sampled_x = sampled_x.cpu().numpy()
sampled_y = sampled_y.cpu().numpy()
train_x = train_x.cpu().numpy()

In [19]:
%%capture
mlab.init_notebook()
v_options = {'mode': 'sphere','scale_factor': 3e-3, 'color': (0, 0, 0)}
vmin, vmax = -2.4, 1.8

### Ground Truth

In [20]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=sampled_y)
cbar = mlab.colorbar(orientation='vertical')
cbar.data_range = (vmin, vmax)
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
# mlab.points3d(test_x[:,0], test_x[:,1], test_x[:,2], **v_options)
mlab.view(0.0,180.0,0.5139171204775793)
mlab.savefig('../outputs/2d_ground_truth.png')

### Posterior Mean Manifold

In [21]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=mean_manifold)
cbar = mlab.colorbar(orientation='vertical')
cbar.data_range = (vmin, vmax)
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
mlab.view(0.0,180.0,0.5139171204775793)
mlab.savefig('../outputs/2d_mean_manifold.png')

### Posterior Mean Vanilla

In [22]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=mean_vanilla)
cbar = mlab.colorbar(orientation='vertical')
cbar.data_range = (vmin, vmax)
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
mlab.view(0.0,180.0,0.5139171204775793)
mlab.savefig('../outputs/2d_mean_vanilla.png')

### Posterior Standard Deviation Manifold

In [23]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=stddev_manifold)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
mlab.view(0.0,180.0,0.5139171204775793)
mlab.savefig('../outputs/2d_std_manifold.png')

### Posterior Standard Deviation Vanilla

In [24]:
mlab.figure(size=(1920, 1360), fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(sampled_x[:, 0], sampled_x[:, 1], sampled_x[:, 2], faces, scalars=stddev_vanilla)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
mlab.view(0.0,180.0,0.5139171204775793)
mlab.savefig('../outputs/2d_std_vanilla.png')